# An Introduction to Hail

Before we can get started with the genomic analysis, we need to first upgrade Databrick's version of Python to 3.6. We'll do so using Anaconda.

First, download the Anaconda3 install script and place it in `/dbfs/tmp/` using the below command.

In [3]:
%sh sudo wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh -O /dbfs/tmp/Anaconda3-5.2.0-Linux-x86_64.sh

Next, run the following two cells to copy the Databrick's python packages to the Databrick's File System (DBFS). We'll use these later.

In [5]:
%sh /databricks/python/bin/pip freeze > /tmp/python_packages.txt

In [6]:
%fs cp file:/tmp/python_packages.txt dbfs:/tmp/python_packages.txt

In the below cell, set the `cluster-name` variable to the name of your cluster, then run the cell. Once complete, click on the cluster's name above (next to the green dot) and restart your cluster. This will install Anaconda, and all of Hail's dependencies. Note that in cluster environments, as long as Python versions between nodes match, it is only necessary to have the Anaconda libraries on the driver.

In [8]:
cluster_name = "mookus"
script = """#!/bin/bash
cp /dbfs/tmp/Anaconda3-5.2.0-Linux-x86_64.sh /tmp
sudo bash /tmp/Anaconda3-5.2.0-Linux-x86_64.sh -b -p /anaconda3
mv /databricks/python /databricks/python_old
ln -s /anaconda3 /databricks/python
cp /dbfs/tmp/python_packages.txt /tmp/python_packages.txt
/databricks/python/bin/pip install --upgrade pip
/databricks/python/bin/pip install -r --no-cache-dir /tmp/python_packages.txt
/databricks/python/bin/conda update conda -y
/databricks/python/bin/conda install -y numpy=1.14.0 pandas=0.22.0 matplotlib=2.2.3 seaborn=0.8.1 bokeh=0.12.13
/databricks/python/bin/pip install parsimonious==0.8.0 ipykernel==4.9.0 decorator==4.3.0
"""
dbutils.fs.put("dbfs:/databricks/init/%s/install_conda.sh" % cluster_name, script, True)

Once the cluster is restarted, run the below cell. If the output displays
```
3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]```
Anaconda was installed properly.

In [10]:
%python
import sys
print (sys.version)

### Getting Started
Before we can begin, let's import the Hail libraries and initialize Hail.

In [12]:
import hail as hl
import hail.expr.aggregators as agg
hl.init(sc)

Before using Hail, we import some standard Python libraries for use throughout the notebook.

In [14]:
from pprint import pprint
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import Span
output_notebook()

###Check tutorial data for download if necessary
This cell downloads the necessary data if it isn’t already present.

In [16]:
hl.utils.get_1kg('/tmp/data')

### Load data from disk
Hail has its own internal data representation, called a MatrixTable. This is both an on-disk file format and a [Python object](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable). Here, we read a MatrixTable from disk.

This dataset was created by downsampling a public 1000 genomes VCF to about 50 MB.

In [18]:
mt = hl.read_matrix_table('/tmp/data/1kg.mt')

### Getting to know our data
It’s important to have easy ways to slice, dice, query, and summarize a dataset. Some of these methods are demonstrated below.

The [rows](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.rows) method can be used to get a table with all the row fields in our MatrixTable.

We can use `rows` along with [select](https://hail.is/docs/0.2/hail.Table.html#hail.Table.select) to pull out 5 variants. The `select` method takes either a string refering to a field name in the table, or a Hail [Expression](https://hail.is/docs/0.2/expr/expression.html#hail.expr.expression.Expression). Here, we leave the arguments blank to keep only the row key fields, `locus` and `alleles`.

Use the `show` method to display the variants.

In [20]:
mt.rows().select().show(5)

Here is how to peek at the first few sample IDs:

In [22]:
mt.s.show(5)

To look at the first few genotype calls, we can use [entries](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.entries) along with `select` and `take`. The `take` method collects the first n rows into a list. Alternatively, we can use the `show` method, which prints the first n rows to the console in a table format.

Try changing `take` to `show` in the cell below.

In [24]:
mt.entry.take(5)

### Adding Column Fields
A Hail MatrixTable can have any number of row fields and column fields for storing data associated with each row and column. Annotations are usually a critical part of any genetic study. Column fields are where you’ll store information about sample phenotypes, ancestry, sex, and covariates. Row fields can be used to store information like gene membership and functional impact for use in QC or analysis.

In this tutorial, we demonstrate how to take a text file and use it to annotate the columns in a MatrixTable.

The file provided contains the sample ID, the population and “super-population” designations, the sample sex, and two simulated phenotypes (one binary, one discrete).

This file can be imported into Hail with [import_table](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_table). This method produces a [Table](https://hail.is/docs/0.2/hail.Table.html#hail.Table) object. Think of this as a Pandas or R dataframe that isn’t limited by the memory on your machine – behind the scenes, it’s distributed with Spark.

In [26]:
table = (hl.import_table('/tmp/data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))

A good way to peek at the structure of a `Table` is to look at its `schema`.

In [28]:
table.describe()

To peek at the first few values, use the `show` method:

In [30]:
table.show(width=100)

Now we’ll use this table to add sample annotations to our dataset, storing the annotations in column fields in our MatrixTable. First, we’ll print the existing column schema:

In [32]:
print(mt.col.dtype)

We use the [annotate_cols](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) method to join the table with the MatrixTable containing our dataset.

In [34]:
mt = mt.annotate_cols(**table[mt.s])

In [35]:
print(mt.col.dtype)

In [36]:
print(mt.col.dtype.pretty())

In [37]:
mt.rows().describe()

In [38]:
mt.cols().describe()

###Query functions and the Hail Expression Language
Hail has a number of useful query functions that can be used for gathering statistics on our dataset. These query functions take Hail Expressions as arguments.

We will start by looking at some statistics of the information in our table. The [aggregate](https://hail.is/docs/0.2/hail.Table.html#hail.Table.aggregate) method can be used to aggregate over rows of the table.

`counter` is an aggregation function that counts the number of occurrences of each unique element. We can use this to pull out the population distribution by passing in a Hail Expression for the field that we want to count by.

In [40]:
pprint(table.aggregate(agg.counter(table.SuperPopulation)))

`stats` is an aggregation function that produces some useful statistics about numeric collections. We can use this to see the distribution of the CaffeineConsumption phenotype.

In [42]:
pprint(table.aggregate(agg.stats(table.CaffeineConsumption)))

However, these metrics aren’t perfectly representative of the samples in our dataset. Here’s why:

In [44]:
table.count()

In [45]:
mt.count_cols()

Since there are fewer samples in our dataset than in the full thousand genomes cohort, we need to look at annotations on the dataset. We can use [aggregate_cols](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.aggregate_cols) to get the metrics for only the samples in our dataset.

In [47]:
mt.aggregate_cols(agg.counter(mt.SuperPopulation))

In [48]:
pprint(mt.aggregate_cols(agg.stats(mt.CaffeineConsumption)))

The functionality demonstrated in the last few cells isn’t anything especially new: it’s certainly not difficult to ask these questions with Pandas or R dataframes, or even Unix tools like `awk`. But Hail can use the same interfaces and query language to analyze collections that are much larger, like the set of variants.

Here we calculate the counts of each of the 12 possible unique SNPs (4 choices for the reference base * 3 choices for the alternate base).

To do this, we need to get the alternate allele of each variant and then count the occurences of each unique ref/alt pair. This can be done with Hail’s `counter` method.

In [50]:
snp_counts = mt.aggregate_rows(agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])))
pprint(snp_counts)

We can list the counts in descending order using Python’s Counter class.

In [52]:
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

It’s nice to see that we can actually uncover something biological from this small dataset: we see that these frequencies come in pairs. C/T and G/A are actually the same mutation, just viewed from from opposite strands. Likewise, T/A and A/T are the same mutation on opposite strands. There’s a 30x difference between the frequency of C/T and A/T SNPs. Why?

The same Python, R, and Unix tools could do this work as well, but we’re starting to hit a wall - the latest [gnomAD release](http://gnomad.broadinstitute.org/) publishes about 250 million variants, and that won’t fit in memory on a single computer.

What about genotypes? Hail can query the collection of all genotypes in the dataset, and this is getting large even for our tiny dataset. Our 284 samples and 10,000 variants produce 10 million unique genotypes. The gnomAD dataset has about **5 trillion** unique genotypes.

Hail plotting functions allow Hail fields as arguments, so we can pass in the DP field directly here. If the range and bins arguments are not set, this function will compute the range based on minimum and maximum values of the field and use the default 50 bins.

In [54]:
from bokeh.plotting import figure
from bokeh.embed import components, file_html
from bokeh.resources import CDN
p = hl.plot.histogram(mt.DP, range=(0,30), bins=30, title='DP Histogram', legend='DP')
html = file_html(p, CDN, "DP Histogram")
displayHTML(html)


### Quality Control
QC is where analysts spend most of their time with sequencing datasets. QC is an iterative process, and is different for every project: there is no “push-button” solution for QC. Each time the Broad collects a new group of samples, it finds new batch effects. However, by practicing open science and discussing the QC process and decisions with others, we can establish a set of best practices as a community.

QC is entirely based on the ability to understand the properties of a dataset. Hail attempts to make this easier by providing the [sample_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) method, which produces a set of useful metrics and stores them in a column field.

In [56]:
mt.col.describe()

In [57]:
mt = hl.sample_qc(mt)

In [58]:
mt.col.describe()

Plotting the QC metrics is a good place to start.

In [60]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
html = file_html(p, CDN, "Call Rate")
displayHTML(html)

In [61]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10,70), legend='Mean Sample GQ')
html = file_html(p, CDN, "Mean Sample GQ")
displayHTML(html)

Often, these metrics are correlated.

In [63]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
html = file_html(p, CDN, "Mean DP vs Call Rate")
displayHTML(html)

Removing outliers from the dataset will generally improve association results. We can draw lines on the above plot to indicate outlier cuts.

In [65]:
p.renderers.extend(
    [Span(location=0.97, dimension='width', line_color='black', line_width=1),
     Span(location=4, dimension='height', line_color='black', line_width=1)])
html = file_html(p, CDN, "Mean DP vs Call Rate")
displayHTML(html)

We’ll want to remove all samples that fall in the bottom left quadrant.

It’s easy to filter when we’ve got the cutoff values decided:

In [67]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % mt.count_cols())

Next is genotype QC. To start, we’ll print the post-sample-QC call rate. It’s actually gone up since the initial summary - dropping low-quality samples disproportionally removed missing genotypes.

Import the `hail.expr.functions` module.

In [69]:
call_rate = mt.aggregate_entries(agg.fraction(hl.is_defined(mt.GT)))
print('pre QC call rate is %.3f' % call_rate)

It’s a good idea to filter out genotypes where the reads aren’t where they should be: if we find a genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error.

In [71]:
ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

mt = mt.filter_entries(filter_condition_ab)

In [72]:
post_qc_call_rate = mt.aggregate_entries(agg.fraction(hl.is_defined(mt.GT)))
print('post QC call rate is %.3f' % post_qc_call_rate)

Variant QC is a bit more of the same: we can use the [variant_qc](https://hail.is/docs/0.2/methods/genetics.html?highlight=variant%20qc#hail.methods.variant_qc) method to produce a variety of useful statistics, plot them, and filter.

In [74]:
mt.row.describe()

In [75]:
mt = hl.variant_qc(mt).cache()

In [76]:
mt.row.describe()

In [77]:
qc_table = mt.rows()
qc_table = qc_table.select(gq_mean = qc_table.variant_qc.gq_stats.mean,
                           call_rate = qc_table.variant_qc.call_rate,
                           p_value_hwe = qc_table.variant_qc.p_value_hwe,
                           AAF = qc_table.variant_qc.AF[1],)

p1 = hl.plot.histogram(qc_table.gq_mean, range=(10,80), legend='Variant Mean GQ')

p2 = hl.plot.histogram(qc_table.AAF, range=(0,1), legend='Alternate Allele Frequency')

p3 = hl.plot.histogram(qc_table.call_rate, range=(.5,1), legend='Variant Call Rate')

p4 = hl.plot.histogram(qc_table.p_value_hwe, range=(0,1), legend='Hardy-Weinberg Equilibrium p-value')

g = gridplot([[p1, p2], [p3, p4]])

html = file_html(g, CDN, "Mean DP vs Call Rate")
displayHTML(html)

These statistics actually look pretty good: we don’t need to filter this dataset. Most datasets require thoughtful quality control, though. The [filter_rows])(https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_rows) method can help!

### Let's do a GWAS
First, we need to restrict to variants that are:

* common (we’ll use a cutoff of 1%)
* uncorrelated (not in linkage disequilibrium)

In [80]:
common_mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

In [81]:
print('Samples: %d  Variants: %d' % (common_mt.count_cols(), common_mt.count_rows()))

These filters removed about 15% of sites (we started with a bit over 10,000). This is NOT representative of most sequencing datasets! We have already downsampled the full thousand genomes dataset to include more common variants than we’d expect by chance.

In Hail, the association tests accept column fields for the sample phenotype and covariates. Since we’ve already got our phenotype of interest (caffeine consumption) in the dataset, we are good to go:

In [83]:
gwas = hl.linear_regression_rows(y=common_mt.CaffeineConsumption, x=common_mt.GT.n_alt_alleles(), covariates=[1.0])
gwas.row.describe()

Looking at the bottom of the above printout, you can see the linear regression adds new row fields for the beta, standard error, t-statistic, and p-value.

Hail makes it easy to make a [Q-Q (quantile-quantile) plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot).

In [85]:
p = hl.plot.qq(gwas.p_value)

html = file_html(p, CDN, "GWAS p-value")
displayHTML(html)

###Confounded!
The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there’s a confounder.

We didn’t tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a [stratified](https://en.wikipedia.org/wiki/Population_stratification) distribution of the phenotype. The solution is to include ancestry as a covariate in our regression.

The [linear_regression](https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression) method can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don’t have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The [pca](https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca) method produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The [hwe_normalized_pca](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca) method does the same, using HWE-normalized genotypes for the PCA.

In [87]:
pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(common_mt.GT)

In [88]:
pprint(pca_eigenvalues)

In [89]:
p = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_mt.cols()[pca_scores.s].SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
html = file_html(p, CDN, "PCA")
displayHTML(html)

Now we can rerun our linear regression, controlling for sample sex and the first few principal components. We’ll do this with input variable the number of alternate alleles as before, and again with input variable the genotype dosage derived from the PL field.

In [91]:
cmt = common_mt.annotate_cols(pca = pca_scores[common_mt.s])

linear_regression_results = hl.linear_regression_rows(
    y=cmt.CaffeineConsumption, x=cmt.GT.n_alt_alleles(),
    covariates=[1.0, cmt.isFemale, cmt.pca.scores[0], cmt.pca.scores[1], cmt.pca.scores[2]])
  
p = hl.plot.qq(linear_regression_results.p_value)
html = file_html(p, CDN, "Q-Q Plot")
displayHTML(html)

We can also use the genotype dosage (the probability-weighted expected number of alternate alleles) instead of the hard call in our regression

In [93]:
def plDosage(pl):
    linear_scaled = pl.map(lambda x: 10 ** (-x / 10))
    pl_sum = hl.sum(linear_scaled)
    normed = linear_scaled / pl_sum
    return 1 * normed[1] + 2 * normed[2]

In [94]:
linear_regression_results = hl.linear_regression_rows(
    y=cmt.CaffeineConsumption, x=plDosage(cmt.PL),
    covariates=[1.0, cmt.isFemale, cmt.pca.scores[0], cmt.pca.scores[1], cmt.pca.scores[2]])

In [95]:
p = hl.plot.qq(linear_regression_results.p_value)
html = file_html(p, CDN, "Q-Q Plot")
displayHTML(html)

That’s more like it! We may not be publishing ten new coffee-drinking loci in Nature, but we shouldn’t expect to find anything but the strongest signals from a dataset of 284 individuals anyway.

###Rare variant analysis
Here we’ll demonstrate how one can use the expression language to group and count by any arbitrary properties in row and column fields. Hail also implements the sequence kernel association test (SKAT).

In [98]:
entries = mt.entries()
results = (entries.group_by(pop = entries.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = agg.count_where(entries.GT.is_het())))

In [99]:
results.show()

What if we want to group by minor allele frequency bin and hair color, and calculate the mean GQ?

In [101]:
entries = entries.annotate(maf_bin = hl.cond(entries.info.AF[0]<0.01, "< 1%",
                             hl.cond(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

results2 = (entries.group_by(af_bin = entries.maf_bin, purple_hair = entries.PurpleHair)
      .aggregate(mean_gq = agg.stats(entries.GQ).mean,
                 mean_dp = agg.stats(entries.DP).mean))

In [102]:
results2.show()

We’ve shown that it’s easy to aggregate by a couple of arbitrary statistics. This specific examples may not provide especially useful pieces of information, but this same pattern can be used to detect effects of rare variation:

* Count the number of heterozygous genotypes per gene by functional category (synonymous, missense, or loss-of-function) to estimate per-gene functional constraint* 
* Count the number of singleton loss-of-function mutations per gene in cases and controls to detect genes involved in disease

###Epilogue
Congrats! You’ve reached the end of the first tutorial. To learn more about Hail’s API and functionality, take a look at the other tutorials. You can check out the [Python API](https://hail.is/docs/0.2/api.html#python-api) for documentation on additional Hail functions. If you use Hail for your own science, we’d love to hear from you on [Zulip chat](https://hail.zulipchat.com/) or the [discussion forum](http://discuss.hail.is/).

### Population Clustering Using K-Means in SparkML
In this section, we'll investigate how to leverage Spark's native machine learning libraries to glean insights from our genomic data. In particular, K-Means clustering will be applied to each sample's principal components and the resulting classifications will be compared to their initial Super Population labeling. 

Hail's API supports exporting objects to the standard Spark ecosystem. The below snippet places our PCA data, sample data, and super population data in a DataFrame.

In [106]:
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, VectorUDT
array_to_vec_udf = udf(lambda a: Vectors.dense(a), VectorUDT())

df = cmt.cols().to_spark().select(col('s'), col('Population'), col('SuperPopulation'), array_to_vec_udf(col('`pca.scores`')).alias('pca'))
df.printSchema()

Next, we'll leverage Spark's ML documentation to build a Machine Learning pipeline that performs K-Means clustering on the PCA vectors. (Note that as this could be accomplished without a pipeline as it is a single stage.) 

Please reference the below documentation on pipelines to see how to assemble the ML pipeline.
https://spark.apache.org/docs/latest/ml-pipeline.html

In [108]:
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans


kmeans = KMeans(k=5, featuresCol="pca", predictionCol='prediction')
pipeline = Pipeline(stages=[kmeans])

Next, we'll fit our DataFrame to the pipline and subsequently generate the K-Means predictions. The rest of the snippet simply converts the resulting Spark DataFrame into a Pandas DataFrame for later visualization.

In [110]:
model = pipeline.fit(df) # Fill in with the code to fit the DataFrame to the
                         # pipeline.
result_df = model.transform(df) # Fill in with the code to 
                                # generate the predictions.
pdf = result_df.select('s', 'SuperPopulation', 'prediction').toPandas()
base_pdf = pdf
pdf['index'] = pdf.index
pdf.SuperPopulation = pd.Categorical(pdf.SuperPopulation)
pdf['SuperPopulationCodes'] = pdf.SuperPopulation.cat.codes
pdf.head()

Finally, run the below cells to generate a stacked bar plot to examine a comparison of how K-means clustered the samples compared to their `SuperPopulations`.

In [112]:
eur0 = pdf[(pdf.SuperPopulation=='EUR') & (pdf.prediction==0)].shape[0]
eur1 = pdf[(pdf.SuperPopulation=='EUR') & (pdf.prediction==1)].shape[0]
eur2 = pdf[(pdf.SuperPopulation=='EUR') & (pdf.prediction==2)].shape[0]
eur3 = pdf[(pdf.SuperPopulation=='EUR') & (pdf.prediction==3)].shape[0]
eur4 = pdf[(pdf.SuperPopulation=='EUR') & (pdf.prediction==4)].shape[0]
eurs = (eur0, eur1, eur2, eur3, eur4)

amr0 = pdf[(pdf.SuperPopulation=='AMR') & (pdf.prediction==0)].shape[0]
amr1 = pdf[(pdf.SuperPopulation=='AMR') & (pdf.prediction==1)].shape[0]
amr2 = pdf[(pdf.SuperPopulation=='AMR') & (pdf.prediction==2)].shape[0]
amr3 = pdf[(pdf.SuperPopulation=='AMR') & (pdf.prediction==3)].shape[0]
amr4 = pdf[(pdf.SuperPopulation=='AMR') & (pdf.prediction==4)].shape[0]
amrs = (amr0, amr1, amr2, amr3, amr4)

eas0 = pdf[(pdf.SuperPopulation=='EAS') & (pdf.prediction==0)].shape[0]
eas1 = pdf[(pdf.SuperPopulation=='EAS') & (pdf.prediction==1)].shape[0]
eas2 = pdf[(pdf.SuperPopulation=='EAS') & (pdf.prediction==2)].shape[0]
eas3 = pdf[(pdf.SuperPopulation=='EAS') & (pdf.prediction==3)].shape[0]
eas4 = pdf[(pdf.SuperPopulation=='EAS') & (pdf.prediction==4)].shape[0]
eass = (eas0, eas1, eas2, eas3, eas4)

afr0 = pdf[(pdf.SuperPopulation=='AFR') & (pdf.prediction==0)].shape[0]
afr1 = pdf[(pdf.SuperPopulation=='AFR') & (pdf.prediction==1)].shape[0]
afr2 = pdf[(pdf.SuperPopulation=='AFR') & (pdf.prediction==2)].shape[0]
afr3 = pdf[(pdf.SuperPopulation=='AFR') & (pdf.prediction==3)].shape[0]
afr4 = pdf[(pdf.SuperPopulation=='AFR') & (pdf.prediction==4)].shape[0]
afrs = (afr0, afr1, afr2, afr3, afr4)

sas0 = pdf[(pdf.SuperPopulation=='SAS') & (pdf.prediction==0)].shape[0]
sas1 = pdf[(pdf.SuperPopulation=='SAS') & (pdf.prediction==1)].shape[0]
sas2 = pdf[(pdf.SuperPopulation=='SAS') & (pdf.prediction==2)].shape[0]
sas3 = pdf[(pdf.SuperPopulation=='SAS') & (pdf.prediction==3)].shape[0]
sas4 = pdf[(pdf.SuperPopulation=='SAS') & (pdf.prediction==4)].shape[0]
sass = (sas0, sas1, sas2, sas3, sas4)

In [113]:
import operator

import numpy as np
import matplotlib.pyplot as plt
plt.close("all")

fig, ax = plt.subplots()

x = np.linspace(0, 4, 5)
width = 0.35

ax.bar(x, eurs, width)
bot=eurs
ax.bar(x, amrs, width, bottom=bot)
bot = tuple(map(operator.add, bot, amrs))
ax.bar(x, eass, width, bottom=bot)
bot = tuple(map(operator.add, bot, eass))
ax.bar(x, afrs, width,bottom=bot)
bot = tuple(map(operator.add, bot, afrs))
ax.bar(x, sass, width, bottom=bot)
ax.set_ylabel('Superpop Counts per Prediction')
ax.set_title('K-Means Clusters vs. SuperPopulations')
ax.set_xticks(x, ('0', '1', '2', '3', '4'))
ax.legend(('EUR', 'AMR', 'EAS', 'AFR', 'SAS'))
display(fig)
